Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.

# Exercise 1: Encrypt an image with AES and ECB mode

In this exercice we want to encrypt an image of Tux, the famous penguin from Linux using the AES algorithm and the Electronic Code Book mode.

<img src="img/ECB_mode.png" style="width:1100px"/>

## Plotting the image of Tux

The file is on `../data/tux.png`. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_array(img: np.ndarray, size = (5, 5)):
    plt.clf()
    fig = plt.figure()
    fig.set_size_inches(size[0], size[1])
    ax = fig.add_axes([0, 0, 1, 1], frameon=False, aspect=1)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(img, cmap='Greys')
    plt.show()

In [ ]:
# open image with PIL.Image
from PIL import Image
img = Image.open("../data/tux.png")

# convert image to numpy ndarray
img = np.array(img)

# plot shape and type of pixels
print(f"Shape of image: {img.shape}")
print(f"Type of image: {img.dtype}")
# show plot of tux!
plot_array(img)

In [ ]:
# save the shape of the image
shape = (img.shape[0], img.shape[1], img.shape[2])

## Converting the image pixels to bytes

In this section you need to convert the `np.array` of the image to an array of bytes that will be our plaintext to encrypt

In [ ]:
# flatten the image
img_flat = img.flatten()

# recall that all pixels are of type unsigned integer of 8 bits (uint8), that's a byte! 
img_bytes = bytes([x for x in img_flat])

# print the first 10 bytes
print(img_bytes[1000:1050])

## Padding the message (image)

We want to use AES with a block size of 16 bytes, let's see if we need to pad the image

In [ ]:
# check if we need to pad the message (image), if so use PKCS7 from cryptography library
block_size = 16
print(f"Bytes remainder {len(img_bytes)%block_size}")

One job less! We don't need to pad the image as it is already a multiple of 16!.

## Declare the AES in ECB mode

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

# draw 32 random bytes for secret key
secret_key = os.urandom(32)

# define cipher with AES algorithm and ECB mode
cipher = Cipher(algorithms.AES(secret_key), modes.ECB(), backend=default_backend())

# define encryptor and decryptor
encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
# encrypt the image
ctx = encryptor.update(img_bytes) + encryptor.finalize()

In [ ]:
print(ctx[0:50])

## Plot the ciphertext

In [ ]:
# reshape the ciphertext to the oringial shape of the image
ctx_flat_int = [x for x in ctx]
ctx_img = np.array(ctx_flat_int).reshape(shape[0], shape[1], shape[2])

# plot ciphertext of Tux!
plot_array(ctx_img)

Well, if an eavesdroper sees this, he certainly knows that this is Tux!. Let's decrypt this and recover the original image 

## Recover the original image

In [ ]:
# get the plaintext in bytes
pltx = decryptor.update(ctx)

# reshape the plaintext to get the pixels
pltx_img = np.array([x for x in pltx], dtype=np.uint8).reshape(shape[0], shape[1], shape[2])

# plot image
plot_array(pltx_img)

# Exercise 2: Encrypt an image with AES and CBC mode

<img src="img/CBC_mode.png" style="width:1100px"/>

In [ ]:
# draw secret key of 32 bytes
secret_key = os.urandom(32)

# draw initialization vector 16 bytes
iv = os.urandom(16)

# build the cipher using CBC
cipher = Cipher(algorithms.AES(secret_key), modes.CBC(iv), backend=default_backend())

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [ ]:
# encrypt image
ctx = encryptor.update(img_bytes) + encryptor.finalize()

In [ ]:
print(ctx[0:100])

In [ ]:
# reshape the ciphertext to the oringial shape of the image and plot
ctx_flat_int = [x for x in ctx]
ctx_img = np.array(ctx_flat_int).reshape(shape[0], shape[1], shape[2])

# plot ciphertext of Tux!
plot_array(ctx_img)

# Conclusions

ECB mode is not a good mode to operate as every byte in the plaintext is converted to the same byte in the ciphertext and therefore it won't work on images. 